# Make RGB NIRCam stamps showing slit footprint
We will be using the `trilogy` package to make RGB stamps and the shutter footprint regions.
The information we need to show the footprint and the source (optional) is all in the "shutters" files.


In [ ]:
from trilogy import trilogy
from astropy.io import fits
from astropy.wcs import WCS
from pdf2image import convert_from_path
from PIL import Image
import astropy.units as u
import glob

import matplotlib
from matplotlib import pyplot as plt
import PIL  # Python Image Library
from PIL import Image
import os, sys
import contextlib

from astropy.nddata import Cutout2D
import numpy as np
from scipy.ndimage import zoom
import pandas as pd
import scipy
import os
import glob
import json
import numpy as np
import tempfile



# Alternative function that produces RGB images from the 5x5" images

In [ ]:
# ---------- SETUP ----------
input_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/"
output_dir = "/home/bpc/University/master/Red_Cardinal/stamps_v4/"
os.makedirs(output_dir, exist_ok=True)

# ---------- STEP 1: Get IDs from NIRCam, PRIMER and COSMOS-Web ----------
f444w_files = glob.glob(os.path.join(input_dir, "*F444W*.fits"))
f444w_ids = [os.path.basename(f).split("_F444W")[0] for f in f444w_files]

f770w_primer_files = glob.glob(os.path.join(input_dir, "*F770W_cutout_primer.fits"))
f770w_primer_ids = [os.path.basename(f).split("_F770W")[0] for f in f770w_primer_files]

f770w_cweb_files = glob.glob(os.path.join(input_dir, "*F770W_cutout_cweb.fits"))
f770w_cweb_ids = [os.path.basename(f).split("_F770W")[0] for f in f770w_cweb_files]

f1800w_files = glob.glob(os.path.join(input_dir, "*F1800W*.fits"))
f1800w_ids = [os.path.basename(f).split("_F1800W")[0] for f in f1800w_files]

# Remove all galaxies in f770w_primer_ids from f770w_cweb_ids
f770w_cweb_ids = np.setdiff1d(f770w_cweb_ids, f770w_primer_ids)

Define the new make_stamp function

In [ ]:
def normalise_image(img, stretch='asinh', Q=10, alpha=1, weight=1.0):
    """
    Normalises the input image with optional stretching and channel weighting.

    Parameters:
    - img: 2D numpy array
    - stretch: Type of stretch ('asinh' or 'linear')
    - Q: Controls asinh stretch strength
    - alpha: Controls non-linearity for asinh
    - weight: Multiplier to boost/dampen this channel’s contribution

    Returns:
    - Normalised image scaled between 0 and 1
    """
    
    # Replace nans, positive and negative infinities with 0.0
    img = np.nan_to_num(img, nan=0.0, posinf=0.0, neginf=0.0)
    
    # Clip possible negative fluxes
    img = np.clip(img, 0, None)
    
    #print(f"Before any scaling: min={np.min(img)}, max={np.max(img)}")
    
    # Subtract the minimum value to shift the range to start from 0
    img_min = np.min(img)
    img -= img_min
    #print(f"After minimum subtraction: min={np.min(img)}, max={np.max(img)}")

    
    #print(f"After weight scaling: min={np.min(img)}, max={np.max(img)}")
    
    # Determine which scaling to us
    if stretch == 'asinh':  # Lupton scaling
        img_scaled = np.arcsinh(alpha * Q * img) / Q
    elif stretch == 'log':
        img_scaled = np.log10(1 + alpha * img)
    elif stretch == 'linear':
        img_scaled = img
    else:
        raise ValueError("Unknown stretch")
        
    #print(f"After stretch: min={np.min(img_scaled)}, max={np.max(img_scaled)}")
    
    # After stretching the image is normalised to 1
    img_scaled = img_scaled / np.nanmax(img_scaled) if np.nanmax(img_scaled) != 0 else img_scaled
    
    #print(f"After normalisation: min={np.min(img_scaled)}, max={np.max(img_scaled)}")
    
    return img_scaled


def preprocess_fits_image(filename, ext=0, stretch='asinh', Q=10, alpha=1, weight=1, normalise=False):
    """
    Load, optionally resample, and normalise a FITS image.

    Parameters:
    - filename: FITS filename with optional extension (e.g., 'file.fits[1]')
    - stretch, Q, alpha, weight: Passed to normalize_image
    - upscale_size: Tuple of (new_y, new_x) size to resize image to

    Returns:
    - Processed 2D numpy image
    """
    try:
        with fits.open(filename) as hdul:
            img = hdul[ext].data.astype(float)
    except Exception as e:
        raise RuntimeError(f"Could not open {filename}[{ext}]: {e}")

    # Remove NaNs and negative values for display purposes
    flat = img.flatten()
    flat = flat[np.isfinite(flat)]  # remove NaNs/Infs

    # Sort pixel values
    sorted_pixels = np.sort(flat)

    # Get the lower X% of the pixels
    cutoff_index = int(len(sorted_pixels) * 0.8)
    faint_pixels = sorted_pixels[:cutoff_index]

    # Compute robust mean
    background_mean = np.mean(faint_pixels)
    background_mean = np.nanmean(img)
    img = np.where(np.isnan(img), background_mean, img)
    img -= np.min(img)
    
    # Ensure all values are non-negative before stretch
    img[img < 0] = 0.0

    # Apply stretch
    if stretch == 'asinh':
        img = np.arcsinh(Q * alpha * img) / Q
    elif stretch == 'linear':
        pass  # No stretch applied
    else:
        raise ValueError(f"Unsupported stretch: {stretch}")

    # Normalise if requested
    if normalise:
        max_val = np.nanmax(img)
        if max_val > 0:
            img /= max_val

    # Clip just in case
    #img = np.clip(img, 0, 1)

    return img
    

def make_stamp(imagesRGB, Q_r, alpha_r, weight_r, Q_g, alpha_g, weight_g, Q_b, alpha_b, weight_b=1.0, stretch='asinh', outfile='stamp.pdf'):
    """
    Make RGB stamp using trilogy
    """
    
    # Parameter dictionary to handle values per channel
    params = {
        'R': {'Q': Q_r, 'alpha': alpha_r, 'weight': weight_r},
        'G': {'Q': Q_g, 'alpha': alpha_g, 'weight': weight_g},
        'B': {'Q': Q_b, 'alpha': alpha_b, 'weight': weight_b}
    }
    
    stretched_images = {}
    global_max = 0.0
    
    # Stretch each image and find the global max
    for colour in ['R', 'G', 'B']:
        image_str = imagesRGB[colour][0]
        fname, ext = image_str.split('[')
        ext = int(ext.replace(']', ''))
        colour_params = params[colour]

        # Stretch but don't normalise yet
        stretched = preprocess_fits_image(
            fname,
            ext,
            stretch=stretch,
            Q=colour_params['Q'],
            alpha=colour_params['alpha'],
            normalise=False  # You'll need to add this option in your function
        )

        stretched_images[colour] = stretched
        max_val = np.nanmax(stretched)
        if max_val > global_max:
            global_max = max_val

    print(f"[INFO] Global max across channels: {global_max}")
    
    # Now normalise to global max
    norm_images = {}
    for colour in ['R', 'G', 'B']:
        norm_images[colour] = stretched_images[colour] / global_max
        norm_images[colour] = np.clip(norm_images[colour], 0, 1)

    # Add scaled images to the dictionary
    temp_files = {}
    for colour in norm_images:
        fname = f'temp_{colour}.fits'
        fits.writeto(fname, norm_images[colour], overwrite=True)
        temp_files[colour] = fname
    
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(norm_images['R'], cmap='Reds', origin='lower')
    ax.axis('off')
    plt.show()
    
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(norm_images['G'], cmap='Greens', origin='lower')
    ax.axis('off')
    plt.show()
    
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(norm_images['B'], cmap='Blues', origin='lower')
    ax.axis('off')
    plt.show()
    
    # Legend info
    legend_text = f"R: {os.path.basename(imagesRGB['R'][0]).split('_')[1]}\n" \
                  f"G: {os.path.basename(imagesRGB['G'][0]).split('_')[1]}\n" \
                  f"B: {os.path.basename(imagesRGB['B'][0]).split('_')[1]}"
    
    for colour in norm_images:
        # Replace remaining NaNs just before stacking
        norm_images[colour] = np.nan_to_num(norm_images[colour], nan=0.0)
        norm_images[colour] = np.clip(norm_images[colour], 0, 1)

    print(norm_images['R'])
    print(norm_images['G'])
    # --- Plot using matplotlib and save ---
    fig, ax = plt.subplots(figsize=(6, 6))
    rgb_image = np.stack([norm_images['R'], norm_images['G'], norm_images['B']], axis=-1)
    ax.imshow(rgb_image, origin='lower')
    #ax.imshow(NIRCam_image, origin='lower')
    ax.text(0.02, 0.98, legend_text, transform=ax.transAxes,
            fontsize=10, va='top', ha='left',
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

    ax.axis('off')  # Hide axes
    
    # Save the RGB image as PDF
    fig.savefig(outfile, bbox_inches='tight', pad_inches=0.0)
    plt.close(fig)  # Clean up the figure


Now let's try and call the function:

In [ ]:
for gal_id in f1800w_ids:
    try:
        print(f"Creating image for {gal_id}...")
        
        # Construct base filenames (without [0])
        r_file_base = os.path.join(input_dir, f"{gal_id}_F1800W_cutout_primer.fits")
        g_file_primer_base = os.path.join(input_dir, f"{gal_id}_F770W_cutout_primer.fits")
        g_file_cweb_base = os.path.join(input_dir, f"{gal_id}_F770W_cutout_cweb.fits")
        b_file_base = os.path.join(input_dir, f"{gal_id}_F444W_cutout_nircam.fits")

        # Decide which F770W file to use
        if gal_id in f770w_primer_ids and os.path.exists(g_file_primer_base):
            g_file = g_file_primer_base + "[1]"
        elif gal_id in f770w_cweb_ids and os.path.exists(g_file_cweb_base):
            g_file = g_file_cweb_base + "[1]"
        else:
            print(f"[Skipping {gal_id}] No valid F770W file found.")
            continue

        # Now safely add [0] to files
        r_file = r_file_base + "[1]"
        b_file = b_file_base + "[0]"
        
        # Stack into imagesRGB dictionary
        imagesRGB = {'R': [r_file], 
                    'G': [g_file], 
                    'B': [b_file]}
        
        # Call your image function (without shutters)
        outfile = os.path.join(output_dir, f"{gal_id}_primer.pdf")
        make_stamp(imagesRGB, outfile=outfile, 
                Q_r=10, alpha_r=1.0, weight_r=1,
                Q_g=10, alpha_g=1.0, weight_g=1,
                Q_b=10, alpha_b=1.0, weight_b=1,
                stretch='asinh')

    except Exception as e:
        print(f"Error processing {gal_id}: {e}")

Produce the stamps for PRIMER 003 F1800W

In [ ]:
for gal_id in f1800w_ids:
    try:
        print(f"Creating image for {gal_id}...")
        
        # Construct base filenames (without [0])
        r_file_base = os.path.join(input_dir, f"{gal_id}_F1800W_cutout_primer_shifted.fits")
        g_file_primer_base = os.path.join(input_dir, f"{gal_id}_F770W_cutout_primer_shifted.fits")
        g_file_cweb_base = os.path.join(input_dir, f"{gal_id}_F770W_cutout_cweb_shifted.fits")
        b_file_base = os.path.join(input_dir, f"{gal_id}_F444W_cutout_nircam.fits")

        # Decide which F770W file to use
        if gal_id in f770w_primer003_ids and os.path.exists(g_file_primer_base):
            g_file = g_file_primer_base + "[1]"
        elif gal_id in f770w_cweb2_ids and os.path.exists(g_file_cweb_base):
            g_file = g_file_cweb_base + "[1]"
        else:
            print(f"[Skipping {gal_id}] No valid F770W file found.")
            #continue

        # Now safely add [0] to files
        r_file = r_file_base + "[1]"
        b_file = b_file_base + "[0]"
        
        # Stack into imagesRGB dictionary
        imagesRGB = {'R': [r_file], 
                    'G': [g_file], 
                    'B': [b_file]}
        
        # Call your image function (without shutters)
        # Q determines the strength of the non-linearity for the stretch.
        # alpha adjusts the sensitivity of the stretch for the given filter.

        outfile = os.path.join(output_dir, f"{gal_id}_primer003.pdf")
        make_stamp(imagesRGB, outfile=outfile)

    except Exception as e:
        print(f"Error processing {gal_id}: {e}")

Produce the stamps for PRIMER 004 F1800W

In [ ]:
for gal_id in f1800w_primer004_ids:
    try:
        #print(f"Creating image for {gal_id}...")
        
        # Construct base filenames (without [0])
        r_file_base = os.path.join(input_dir, f"{gal_id}_F1800W_cutout_primer004_shifted.fits")
        b_file_primer_base = os.path.join(input_dir, f"{gal_id}_F770W_cutout_primer004_shifted.fits")
        b_file_cweb_base = os.path.join(input_dir, f"{gal_id}_F770W_cutout_cweb*_shifted.fits")

        # Decide which F770W file to use
        if gal_id in f1800w_primer004_ids and os.path.exists(b_file_primer_base):
            b_file = b_file_primer_base + "[1]"
        elif gal_id in f770w_cweb1_ids and os.path.exists(b_file_cweb_base):
            b_file = b_file_cweb_base + "[1]"
            print("Found a galaxy that is mapped by PRIMER 004 and COSMOS-Web 1: ", gal_id)
        elif gal_id in f770w_cweb1_ids and os.path.exists(b_file_cweb_base):
            print("Found a galaxy that is mapped by PRIMER 004 and COSMOS-Web 2: ", gal_id)
        else:
            print(f"[Skipping {gal_id}] No valid F770W file found.")
            continue
        
        # Now safely add [0] to files
        r_file = r_file_base + "[1]"
        with fits.open(r_file_base) as hdul:
            r_data = hdul[1].data
            g_data = np.zeros_like(r_data)  # fake green
        fits.writeto('fake_green.fits', g_data, overwrite=True)
        
        # Stack into imagesRGB dictionary
        imagesRGB = {'R': [r_file], 
                    'G': ['fake_green.fits[0]'], 
                    'B': [b_file]}
        
        # Call your image function (without shutters)
        # Q determines the strength of the non-linearity for the stretch.
        # alpha adjusts the sensitivity of the stretch for the given filter.

        outfile = os.path.join(output_dir, f"{gal_id}_primer004.pdf")
        make_stamp(imagesRGB, outfile=outfile)

    except Exception as e:
        print(f"Error processing {gal_id}: {e}")


Produce stamps for COSMOS-Web in the F770W filters (Obs 1&2)

In [ ]:
for gal_id in f770w_cweb1_ids:
    try:
        print(f"Creating image for {gal_id}...")#
        
        # Construct base filenames (without [0])
        r_file_base = os.path.join(input_dir, f"{gal_id}_F770W_cutout_cweb1_shifted.fits")
        
        # Now safely add [1] to files
        r_file = r_file_base + "[1]"
        with fits.open(r_file_base) as hdul:
            r_data = hdul[1].data
            g_data = np.zeros_like(r_data)  # fake green
            b_data = np.zeros_like(r_data)  # fake blue
        fits.writeto('fake_green.fits', g_data, overwrite=True)
        fits.writeto('fake_blue.fits', b_data, overwrite=True)
        
        # Stack into imagesRGB dictionary
        imagesRGB = {'R': [r_file], 
                    'G': ['fake_green.fits[0]'], 
                    'B': ['fake_blue.fits[0]']}
        
        # Call your image function (without shutters)
        # Q determines the strength of the non-linearity for the stretch.
        # alpha adjusts the sensitivity of the stretch for the given filter.

        outfile = os.path.join(output_dir, f"{gal_id}_cweb1.pdf")
        make_stamp(imagesRGB, outfile=outfile)

    except Exception as e:
        print(f"Error processing {gal_id}: {e}")

for gal_id in f770w_cweb2_ids:
    try:
        print(f"Creating image for {gal_id}...")
        
        # Construct base filenames (without [0])
        r_file_base = os.path.join(input_dir, f"{gal_id}_F770W_cutout_cweb2_shifted.fits")
        
        # Now safely add [1] to files
        r_file = r_file_base + "[1]"
        with fits.open(r_file_base) as hdul:
            r_data = hdul[1].data
            g_data = np.zeros_like(r_data)  # fake green
            b_data = np.zeros_like(r_data)  # fake blue
        fits.writeto('fake_green.fits', g_data, overwrite=True)
        fits.writeto('fake_blue.fits', b_data, overwrite=True)
        
        # Stack into imagesRGB dictionary
        imagesRGB = {'R': [r_file], 
                    'G': ['fake_green.fits[0]'], 
                    'B': ['fake_blue.fits[0]']}
        
        # Call your image function (without shutters)
        # Q determines the strength of the non-linearity for the stretch.
        # alpha adjusts the sensitivity of the stretch for the given filter.

        outfile = os.path.join(output_dir, f"{gal_id}_cweb2.pdf")
        make_stamp(imagesRGB, outfile=outfile)

    except Exception as e:
        print(f"Error processing {gal_id}: {e}")




Check extensions in cutout FITS files

In [ ]:
r_file = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/18769_F1800W_cutout_primer.fits"
with fits.open(r_file) as hdul:
    for i, hdu in enumerate(hdul):
        print(f"HDU {i}: {hdu.name}, shape = {getattr(hdu.data, 'shape', None)}")

g_file = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/18769_F770W_cutout_primer.fits"
with fits.open(g_file) as hdul:
    for i, hdu in enumerate(hdul):
        print(f"HDU {i}: {hdu.name}, shape = {getattr(hdu.data, 'shape', None)}")
        
b_file = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/18769_F444W_cutout_nircam.fits"
with fits.open(b_file) as hdul:
    for i, hdu in enumerate(hdul):
        print(f"HDU {i}: {hdu.name}, shape = {getattr(hdu.data, 'shape', None)}")
